Imports

In [1]:
from configParser import ConfigParser, getModelName, getDatasetName

Parameters

In [2]:
experimentName = "test_22987_suffix50_11_CNN2_differentConfig_softmax2"
numOfExperiments=100

params = {
    # Do not change for multi/hyperp experiments
    "image_path": "/data/BGNN_data/INHS_cropped",
    "suffix": '50_11', # used to get a subset cleaned_metadata file. set to None otherwise to use full metadata
    
    # dataset
    "training_count":0.64,
    "validation_count":0.16,
    
    # image
    "imageDimension": 224, # Deprecated and should not be changed from 224
    "n_channels": 3,
    "augmentation": False, # False is well better for Hierarchy

    # training
    "batchSize":550, # Bigger is more stable
    "n_epochs":1000,
    "learning_rate":0.01, # Seems 0.0001 to be best value with Adam. 0.1 best for SGD
    "numOfTrials":3,
    "patience": 100,
    "useHeirarchy": True, #True: This makes a big good difference!
    # temp. Should be optimized and removed
    "useRelu": True, # True slows it down with no benefit,
    "downsample":True, #False,
    "downsampleOutput":50, #50 is less average time but also less stability, keep 50
    "takeFromIntermediate":True, #True,
    "takeFromIntermediateOutput":50, #50, keep 50
    "fc_layers": 2, # 2 seems to be best. definitely better than 1
    "useAdam": False, # Keep it to True for now!
    "resnet": "18", # Keep '18'. '50' is not bad but not as good.
    "normalize": False, # Does not seem to matter with no augmentation. Keep False for now
    "softmax": True, # bI would keep false
}


Create file

In [3]:
cp = ConfigParser(experimentName)

cp.write(params)

'test_22987_suffix50_11_CNN2_differentConfig_softmax2/params.json'

Read file

In [4]:
j = cp.read()
print(j)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 1000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'useRelu': True, 'downsample': True, 'downsampleOutput': 50, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'fc_layers': 2, 'useAdam': False, 'resnet': '18', 'normalize': False, 'softmax': True}


Create hyperp

In [5]:
hyperp_params = {
#     "dummy":[0]
#     "fc_layers":[1,2,3]
#     "imageDimension": [28, 56, 224],
#     "n_channels": [1, 3],
#     "kernelSize": [5, 9],
#     "useZCAWhitening": [False, True],
#     "patience": [50, 100],
#     "batchSize": [50, 100, 200, 550],
#     "useHeirarchy": [True, False],
#     "resnet": ['18', '50']
    "softmax": [False, True]
}

cp.writeHyperp(hyperp_params)

'test_22987_suffix50_11_CNN2_differentConfig_softmax2/hyperp_params.json'

Get experiments

In [6]:
for i in cp.getHyperpIter():
    print(i)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 1000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'useRelu': True, 'downsample': True, 'downsampleOutput': 50, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'fc_layers': 2, 'useAdam': False, 'resnet': '18', 'normalize': False, 'softmax': False}
{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 1000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'useRelu': True, 'downsample': True, 'downsampleOutput': 50, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'fc_layers': 2, 'useAdam': False, 'resnet': '18', 'normalize': False, 's

Model name

In [7]:
print(getModelName(j))
print(getDatasetName(j))

tc0.640000_vc0.160000_d224_c3_augFalse_nFalse_bs550_e1000_p100_lr0.010000_hTrue_reluTrue_dsTrue_dso50_tfiTrue_tfio50_admFalse_res18_fc2_smaxTrue
tc0.640000_vc0.160000_d224_c3_augFalse_nFalse


Hyperp search write

In [8]:
from hyperopt import fmin, hp, rand, tpe
from hyperopt.pyll.base import scope
import math

useRandomSearch = True
hyperp_search_params = {
#     "kernels": [ scope.int(hp.quniform('kernels1', 8, 16, 1)),
#                 scope.int(hp.quniform('kernels2', 16, 32, 1)),
#                 scope.int(hp.qloguniform('kernels3', math.log(1), math.log(16), 1)),
#                 scope.int(hp.qloguniform('kernels4', math.log(1), math.log(16), 1)),
#                 scope.int(hp.qloguniform('kernels5', math.log(1), math.log(16), 1))],
#     "kernelSize": scope.int(hp.quniform('kernelSize', 3, 10, 1)), 
    # "useZCAWhitening": hp.choice('useZCAWhitening', [False, True]), 
#     "n_channels": scope.int(hp.choice('n_channels', [1, 3])), 
#     "imageDimension": scope.int(hp.quniform('imageDimension', 28, 420, 28)),
    "batchSize": scope.int(hp.quniform('batchSize', 50, 550, 50)),
    "learning_rate": hp.loguniform('learning_rate', math.log(0.0001), math.log(0.01)),
    "patience": scope.int(hp.quniform('patience', 50, 500, 25)),
}

cp.writeHyperpSearch(hyperp_search_params)

'test_22987_suffix50_11_CNN2_differentConfig_softmax2/hyperp_search_params.pkl'

Hyperp search read

In [9]:
hyperp_obj = cp.getHyperpSearchObject()
print(hyperp_obj)

def objective(experiment_params):
    print(experiment_params)
    dummyObjective = 1
    return dummyObjective

fmin(objective,
     space=hyperp_obj,
     algo=rand.suggest, 
     max_evals=numOfExperiments)

{'batchSize': <hyperopt.pyll.base.Apply object at 0x7f1d4b5bed68>, 'learning_rate': <hyperopt.pyll.base.Apply object at 0x7f1d4b5434a8>, 'patience': <hyperopt.pyll.base.Apply object at 0x7f1d4b543588>, 'image_path': <hyperopt.pyll.base.Apply object at 0x7f1d4b5becf8>, 'suffix': <hyperopt.pyll.base.Apply object at 0x7f1d4b543860>, 'training_count': <hyperopt.pyll.base.Apply object at 0x7f1d4b5439b0>, 'validation_count': <hyperopt.pyll.base.Apply object at 0x7f1d4b543b00>, 'imageDimension': <hyperopt.pyll.base.Apply object at 0x7f1d4b543c50>, 'n_channels': <hyperopt.pyll.base.Apply object at 0x7f1d4b543da0>, 'augmentation': <hyperopt.pyll.base.Apply object at 0x7f1d4b543ef0>, 'n_epochs': <hyperopt.pyll.base.Apply object at 0x7f1d4b548080>, 'numOfTrials': <hyperopt.pyll.base.Apply object at 0x7f1d4b5481d0>, 'useHeirarchy': <hyperopt.pyll.base.Apply object at 0x7f1d4b548320>, 'useRelu': <hyperopt.pyll.base.Apply object at 0x7f1d4b548470>, 'downsample': <hyperopt.pyll.base.Apply object at 0

{'augmentation': False, 'batchSize': 250, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0002179200254930283, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 400, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': True, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchSize': 500, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00026208474658322807, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 250, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': 

{'augmentation': False, 'batchSize': 400, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00208593486008852, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 125, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': True, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchSize': 200, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00035274806842544766, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 175, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': Tr

{'augmentation': False, 'batchSize': 250, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0015373416927708008, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 350, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': True, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchSize': 300, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.003099057842200713, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 300, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': Tr

{'augmentation': False, 'batchSize': 200, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00038259646628794637, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 350, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': True, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchSize': 500, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00020314667515884487, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 450, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy':

{'augmentation': False, 'batchSize': 150, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00022561440442391784, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 75, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': True, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchSize': 400, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0037166486368849175, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 300, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': T

{'augmentation': False, 'batchSize': 550, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00165260325876852, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 225, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': True, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchSize': 500, 'downsample': True, 'downsampleOutput': 50, 'fc_layers': 2, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00018063328935848775, 'n_channels': 3, 'n_epochs': 1000, 'normalize': False, 'numOfTrials': 3, 'patience': 75, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': Tru

{'augmentation': 0,
 'batchSize': 100.0,
 'downsample': 0,
 'downsampleOutput': 0,
 'fc_layers': 0,
 'imageDimension': 0,
 'image_path': 0,
 'learning_rate': 0.004595362493294816,
 'n_channels': 0,
 'n_epochs': 0,
 'normalize': 0,
 'numOfTrials': 0,
 'patience': 100.0,
 'resnet': 0,
 'softmax': 0,
 'suffix': 0,
 'takeFromIntermediate': 0,
 'takeFromIntermediateOutput': 0,
 'training_count': 0,
 'useAdam': 0,
 'useHeirarchy': 0,
 'useRelu': 0,
 'validation_count': 0}

Selected hyperp write

In [10]:
selected_hyperp_params = [
    {"downsampleOutput":50, "takeFromIntermediateOutput":50,},
    {"downsampleOutput":100, "takeFromIntermediateOutput":100,},
    {"downsampleOutput":200, "takeFromIntermediateOutput":200,},
    {"downsampleOutput":300, "takeFromIntermediateOutput":300,},
]

cp.writeHyperpSelected(selected_hyperp_params)

'test_22987_suffix50_11_CNN2_differentConfig_softmax2/hyperp_selected_params.pkl'

Selected hyperp read

In [11]:
for i in cp.getHyperpSelectedIter():
    print(i)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 1000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'useRelu': True, 'downsample': True, 'downsampleOutput': 50, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'fc_layers': 2, 'useAdam': False, 'resnet': '18', 'normalize': False, 'softmax': True}
{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 1000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': True, 'useRelu': True, 'downsample': True, 'downsampleOutput': 100, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 100, 'fc_layers': 2, 'useAdam': False, 'resnet': '18', 'normalize': False, '